In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import matplotlib.pyplot as plt
import linear_shrinkage
import Util
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os

In [2]:
import scipy.stats

In [3]:
ret, ret_nonan, univ, tradeidx, dates = Util.load_data()

In [22]:
scipy.stats.t.fit(ret[:, 2353])

(2.3401105566945226, 0.00020257082567982178, 0.014055176353385983)

In [15]:
for i, c in enumerate(ret.T):
    if np.isnan(c).sum() == 0:
        print(i, end = " ")

59 73 96 100 130 148 162 174 176 178 188 191 196 221 224 247 253 254 257 264 268 269 272 274 275 285 286 289 292 298 305 308 310 315 325 327 332 339 340 341 346 348 350 357 358 369 371 378 379 383 386 388 404 406 407 411 416 420 432 440 445 446 453 462 463 466 467 468 479 482 483 486 487 496 497 501 506 507 509 513 516 526 528 530 532 533 543 545 546 549 552 554 556 559 562 563 566 570 575 577 579 582 588 589 591 602 614 615 616 625 631 635 638 644 645 651 655 671 687 692 705 709 717 718 724 728 731 736 737 738 741 743 747 749 769 777 783 793 810 812 831 833 834 839 842 846 849 858 866 871 872 876 883 895 899 900 908 918 919 922 923 931 936 940 948 951 955 959 962 977 981 986 1003 1004 1010 1013 1017 1018 1025 1029 1036 1040 1042 1050 1060 1071 1073 1082 1095 1106 1133 1137 1138 1139 1148 1151 1156 1166 1168 1170 1172 1173 1174 1176 1183 1185 1191 1204 1206 1207 1214 1218 1219 1222 1226 1233 1234 1237 1243 1257 1273 1287 1301 1305 1312 1313 1328 1352 1361 1378 1381 1382 1386 1387 1395 

In [16]:
pastRet = Util.get_past_period(5, 1260, 50, univ, tradeidx, ret)

In [34]:
def kendall_cov_mat(data):
    df = pd.DataFrame(data)
    kendall_corr_mat = df.corr(method='kendall').values
    corr_mat = np.sin(0.5 * np.pi * kendall_corr_mat)
    stdmat = np.diag(np.sqrt(np.diag(np.cov(data.T))))
    return stdmat.dot(corr_mat).dot(stdmat)

In [36]:
np.cov(pastRet.T)

array([[2.05829217e-04, 1.01374936e-04, 7.79024701e-05, ...,
        7.16199455e-05, 4.26593893e-05, 9.22385255e-05],
       [1.01374936e-04, 3.16480616e-04, 1.21236753e-04, ...,
        9.32632075e-05, 6.83876301e-05, 1.78614446e-04],
       [7.79024701e-05, 1.21236753e-04, 3.03730569e-04, ...,
        7.05378656e-05, 4.45922038e-05, 1.28736925e-04],
       ...,
       [7.16199455e-05, 9.32632075e-05, 7.05378656e-05, ...,
        3.61465762e-04, 5.46734048e-05, 6.57741039e-05],
       [4.26593893e-05, 6.83876301e-05, 4.45922038e-05, ...,
        5.46734048e-05, 2.70215089e-04, 8.23194401e-05],
       [9.22385255e-05, 1.78614446e-04, 1.28736925e-04, ...,
        6.57741039e-05, 8.23194401e-05, 7.45122812e-04]])

In [35]:
kendall_cov_mat(pastRet)

array([[2.05829217e-04, 1.02935313e-04, 8.23560386e-05, ...,
        8.09504817e-05, 4.71152149e-05, 8.75975909e-05],
       [1.02935313e-04, 3.16480616e-04, 1.21069728e-04, ...,
        9.53693963e-05, 6.62012243e-05, 1.62316554e-04],
       [8.23560386e-05, 1.21069728e-04, 3.03730569e-04, ...,
        8.10790863e-05, 5.41646975e-05, 1.32807596e-04],
       ...,
       [8.09504817e-05, 9.53693963e-05, 8.10790863e-05, ...,
        3.61465762e-04, 5.41552822e-05, 6.33425005e-05],
       [4.71152149e-05, 6.62012243e-05, 5.41646975e-05, ...,
        5.41552822e-05, 2.70215089e-04, 8.36091083e-05],
       [8.75975909e-05, 1.62316554e-04, 1.32807596e-04, ...,
        6.33425005e-05, 8.36091083e-05, 7.45122812e-04]])

In [17]:
data = pastRet

In [18]:
print(data.shape)

(1260, 50)


In [19]:
df = pd.DataFrame(data)
kendal_corr_mat = df.corr(method='kendall').values
corr_mat = np.sin(.5 * np.pi * kendal_corr_mat)

In [20]:
print(corr_mat)

[[1.         0.40330845 0.32938035 ... 0.29677844 0.19978039 0.22367875]
 [0.40330845 1.         0.39049705 ... 0.28196934 0.2263798  0.33425301]
 [0.32938035 0.39049705 1.         ... 0.24469833 0.18906767 0.27916742]
 ...
 [0.29677844 0.28196934 0.24469833 ... 1.         0.1732816  0.12205278]
 [0.19978039 0.2263798  0.18906767 ... 0.1732816  1.         0.18633089]
 [0.22367875 0.33425301 0.27916742 ... 0.12205278 0.18633089 1.        ]]


In [21]:
print(np.corrcoef(data.T))

[[1.         0.39719477 0.31156844 ... 0.26257108 0.18088657 0.23552928]
 [0.39719477 1.         0.39103577 ... 0.27574218 0.23385637 0.36781471]
 [0.31156844 0.39103577 1.         ... 0.21288471 0.15565386 0.27061069]
 ...
 [0.26257108 0.27574218 0.21288471 ... 1.         0.17493944 0.12673817]
 [0.18088657 0.23385637 0.15565386 ... 0.17493944 1.         0.18345674]
 [0.23552928 0.36781471 0.27061069 ... 0.12673817 0.18345674 1.        ]]


In [31]:
stdmat = np.diag(np.sqrt(np.diag(np.cov(data.T))))

In [32]:
stdmat.dot(corr_mat).dot(stdmat)

array([[2.05829217e-04, 1.02935313e-04, 8.23560386e-05, ...,
        8.09504817e-05, 4.71152149e-05, 8.75975909e-05],
       [1.02935313e-04, 3.16480616e-04, 1.21069728e-04, ...,
        9.53693963e-05, 6.62012243e-05, 1.62316554e-04],
       [8.23560386e-05, 1.21069728e-04, 3.03730569e-04, ...,
        8.10790863e-05, 5.41646975e-05, 1.32807596e-04],
       ...,
       [8.09504817e-05, 9.53693963e-05, 8.10790863e-05, ...,
        3.61465762e-04, 5.41552822e-05, 6.33425005e-05],
       [4.71152149e-05, 6.62012243e-05, 5.41646975e-05, ...,
        5.41552822e-05, 2.70215089e-04, 8.36091083e-05],
       [8.75975909e-05, 1.62316554e-04, 1.32807596e-04, ...,
        6.33425005e-05, 8.36091083e-05, 7.45122812e-04]])

In [33]:
np.cov(data.T)

array([[2.05829217e-04, 1.01374936e-04, 7.79024701e-05, ...,
        7.16199455e-05, 4.26593893e-05, 9.22385255e-05],
       [1.01374936e-04, 3.16480616e-04, 1.21236753e-04, ...,
        9.32632075e-05, 6.83876301e-05, 1.78614446e-04],
       [7.79024701e-05, 1.21236753e-04, 3.03730569e-04, ...,
        7.05378656e-05, 4.45922038e-05, 1.28736925e-04],
       ...,
       [7.16199455e-05, 9.32632075e-05, 7.05378656e-05, ...,
        3.61465762e-04, 5.46734048e-05, 6.57741039e-05],
       [4.26593893e-05, 6.83876301e-05, 4.45922038e-05, ...,
        5.46734048e-05, 2.70215089e-04, 8.23194401e-05],
       [9.22385255e-05, 1.78614446e-04, 1.28736925e-04, ...,
        6.57741039e-05, 8.23194401e-05, 7.45122812e-04]])

In [40]:
import scipy
scipy.io.loadmat('matlab/data/historical_N_500_T_125/257.mat')['S'].shape

(500, 500)